In [ ]:
!pip install beautifulsoup4 requests spacy textblob scikit-learn

# Modelo spaCy en español
!python -m spacy download es_core_news_md

import requests
from bs4 import BeautifulSoup
import spacy
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

import re, json, datetime
from dataclasses import dataclass, field
from typing import List, Dict, Any

nlp = spacy.load("es_core_news_md")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 19.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install google-genai
#!pip install openai

##LLM Gemini

In [ ]:
import os

# ⚠️ SOLO para uso personal; evita subir esto a ningún repositorio
os.environ["GEMINI_API_KEY"] = "Aqui va la llave de google"

In [ ]:
from google import genai

# Crea el cliente usando la variable de entorno GEMINI_API_KEY
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

In [ ]:
def call_llm(prompt: str, model_name: str = "gemini-2.5-flash") -> str:
     """
     Envía un prompt de texto al modelo Gemini y devuelve la respuesta en texto plano.
     - prompt: instrucción o contexto que le pasas al LLM.
     - model_name: puedes usar gemini-2.5-flash (rápido) o gemini-2.5-pro (más potente).
     """
     response = client.models.generate_content(
         model=model_name,
         contents=prompt
     )
    # La respuesta ya viene estructurada; text te da todo concatenado
     return response.text

In [ ]:
test_prompt = "Explica en 3 frases cuál es el objetivo de un sistema multi-agente para análisis de reseñas de smartphones."
respuesta = call_llm(test_prompt)
print(respuesta)

El objetivo principal es extraer de forma eficiente y exhaustiva información valiosa de grandes volúmenes de reseñas de smartphones. Para ello, el sistema distribuye tareas especializadas entre agentes autónomos que colaboran, como identificar sentimientos, extraer características o detectar problemas comunes. Así, se logra una comprensión profunda y multidimensional de las preferencias y experiencias de los usuarios, imposible de alcanzar con métodos tradicionales.


In [ ]:
@dataclass
class MASState:
    query: str
    urls: List[Dict[str, Any]] = field(default_factory=list)
    documents: List[Dict[str, Any]] = field(default_factory=list)
    nlp_results: Dict[str, Any] = field(default_factory=dict)
    factcheck_results: List[Dict[str, Any]] = field(default_factory=list)
    created_at: str = field(default_factory=lambda: datetime.datetime.now().isoformat())

class BaseAgent:
    def __init__(self, name: str):
        self.name = name

    def run(self, state: MASState) -> MASState:
        raise NotImplementedErro

#Agente de Busqueda Web

In [ ]:
class SmartphoneWebSearchAgent(BaseAgent):
    """
    Agente de búsqueda que devuelve URLs de artículos de análisis de smartphones,
    no páginas de índice.
    """

    def __init__(self, name="smartphone_search", max_results=5):
        super().__init__(name)
        self.max_results = max_results

    def run(self, state: MASState) -> MASState:
        # EJEMPLO: artículos concretos (ajusta las URLs a las que tú quieras analizar)
        example_urls = [
            {
                "url": "https://www.xataka.com/moviles/samsung-galaxy-s24-ultra-analisis-caracteristicas-precio-especificaciones",
                "title": "Análisis Samsung Galaxy S24 Ultra",
                "source": "Xataka"
            },
            {
                "url": "https://www.xataka.com/analisis/apple-iphone-15-pro-max-analisis-caracteristicas-precio-especificaciones",
                "title": "Análisis iPhone 15 Pro Max",
                "source": "Xataka"
            },
            {
                "url": "https://www.xataka.com/moviles/xiaomi-14-analisis-caracteristicas-precio-especificaciones",
                "title": "Análisis Xiaomi 14",
                "source": "Xataka"
            }
        ]

        state.urls = example_urls[:self.max_results]
        print(f"[{self.name}] Se seleccionaron {len(state.urls)} artículos concretos.")
        return state

#Agente Scraper

In [ ]:
class ScraperAgent(BaseAgent):
    """
    Agente encargado de descargar y limpiar el texto de las páginas web.
    Intenta centrarse en el cuerpo del artículo.
    """

    def __init__(self, name="scraper"):
        super().__init__(name)
        self.headers = {
            "User-Agent": (
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/122.0.0.0 Safari/537.36"
            )
        }

    def extract_main_text(self, soup: BeautifulSoup) -> str:
        # 1. Intentar con la etiqueta <article>
        article = soup.find("article")
        if article:
            text = article.get_text(separator=" ", strip=True)
            if len(text.split()) > 100:
                return text

        # 2. Intentar con contenedores comunes
        candidates = []
        for selector in ["main", "div#content", "div.article-body", "div.entry-content"]:
            node = soup.select_one(selector)
            if node:
                t = node.get_text(separator=" ", strip=True)
                candidates.append(t)

        candidates = [c for c in candidates if len(c.split()) > 100]
        if candidates:
            # Devolver el texto más largo de los candidatos
            return max(candidates, key=len)

        # 3. Fallback: texto completo de la página (limpiando scripts, etc.)
        for tag in soup(["script", "style", "noscript"]):
            tag.decompose()
        text = soup.get_text(separator=" ", strip=True)
        return text

    def run(self, state: MASState) -> MASState:
        docs = []
        for meta in state.urls:
            url = meta["url"]
            try:
                print(f"[{self.name}] Descargando: {url}")
                resp = requests.get(url, headers=self.headers, timeout=15)
                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, "html.parser")
                    text = self.extract_main_text(soup)

                    # Filtro: descartar textos muy cortos
                    if len(text.split()) < 150:
                        print(f"  - Advertencia: texto muy corto en {url} ({len(text.split())} palabras). Se descarta.")
                        continue

                    docs.append({
                        "url": url,
                        "source": meta.get("source", ""),
                        "title": meta.get("title", ""),
                        "text": text
                    })
                else:
                    print(f"  - Error HTTP {resp.status_code} en {url}")
            except Exception as e:
                print(f"  - Error al acceder a {url}: {e}")

        state.documents = docs
        print(f"[{self.name}] Se obtuvieron {len(docs)} documentos con texto suficiente.")
        return state

#Agente NLP (NER + Sentimiento + Clustering)

In [ ]:
class SmartphoneNLPAgent(BaseAgent):
    """
    Agente que realiza:
    - Extracción de entidades (NER)
    - Análisis de sentimiento
    - Clustering de temas (TF-IDF + KMeans)
    """

    def __init__(self, name="smartphone_nlp", n_clusters=3):
        super().__init__(name)
        self.n_clusters = n_clusters

    def extract_entities(self, text: str):
        doc = nlp(text)
        ents = [(ent.text, ent.label_) for ent in doc.ents]
        return ents

    def sentiment_score(self, text: str) -> float:
        blob = TextBlob(text)
        return blob.sentiment.polarity

    def cluster_topics(self, docs: List[str]):
        """
        Devuelve:
        - labels: lista con el id de cluster por documento
        - top_terms: términos representativos por cluster
        """
        if len(docs) < 2:
            # Con un solo documento no tiene sentido agrupar: todo a cluster 0
            return [0] * len(docs), {}

        try:
            vectorizer = TfidfVectorizer(max_features=3000)
            X = vectorizer.fit_transform(docs)
            k = min(self.n_clusters, len(docs))
            model = KMeans(n_clusters=k, random_state=42, n_init="auto")
            labels = model.fit_predict(X)
            # Convertimos el array a lista para evitar problemas de truth value
            labels = list(labels)

            terms = vectorizer.get_feature_names_out()
            top_terms = {}
            for cluster_id in range(k):
                center = model.cluster_centers_[cluster_id]
                top_idx = center.argsort()[-10:]
                top_terms[cluster_id] = [terms[i] for i in top_idx]

            return labels, top_terms

        except Exception as e:
            print(f"[{self.name}] Error en cluster_topics: {e}")
            return [0] * len(docs), {}

    def run(self, state: MASState) -> MASState:
        if not state.documents:
            print(f"[{self.name}] No hay documentos para procesar NLP.")
            state.nlp_results = {"docs": [], "cluster_terms": {}}
            return state

        print(f"[{self.name}] Iniciando análisis NLP sobre {len(state.documents)} documentos...")

        # Textos recortados
        texts = []
        for d in state.documents:
            text = d.get("text", "") or ""
            texts.append(text[:4000])

        labels, top_terms = self.cluster_topics(texts)

        doc_results = []
        for i, d in enumerate(state.documents):
            raw_text = d.get("text", "") or ""
            snippet = raw_text[:2000]

            ents = self.extract_entities(snippet) if snippet else []
            sent = self.sentiment_score(snippet) if snippet else 0.0
            if labels is not None and len(labels) > i:
                cluster = int(labels[i])
            else:
                cluster = 0

            doc_results.append({
                "url": d.get("url", ""),
                "source": d.get("source", ""),
                "title": d.get("title", ""),
                "sentiment": sent,
                "cluster": cluster,
                "entities": ents
            })

        state.nlp_results = {
            "docs": doc_results,
            "cluster_terms": top_terms
        }

        print(f"[{self.name}] Análisis NLP completado.")
        return state

#Agente FactCheck

In [ ]:
class FactCheckAgent(BaseAgent):
    """
    Agente que extrae afirmaciones verificables del texto y las marca
    como 'Requiere verificación adicional'.
    Versión robusta: solo actúa si hay texto suficiente y filtra respuestas meta del LLM.
    """

    def __init__(self, name="fact_checker", max_claims=3):
        super().__init__(name)
        self.max_claims = max_claims

    def extract_claims_with_llm(self, text: str, n: int = 3):
        """
        Usa el LLM para extraer hasta n afirmaciones verificables sobre smartphones.
        Si el texto está vacío, devuelve lista vacía.
        """
        if not text or not text.strip():
            # No hay texto para analizar
            return []

        prompt = f"""
        Extrae hasta {n} afirmaciones verificables sobre smartphones
        (hechos concretos, medibles u objetivamente comprobables) del siguiente texto.
        Devuélvelas como una lista numerada, una afirmación por línea.
        No escribas explicaciones adicionales, solo las afirmaciones.

        Texto:
        {text}
        """

        try:
            resp = call_llm(prompt)
            # Separar por líneas y limpiar
            raw_lines = [c.strip("- ").strip() for c in resp.split("\n") if c.strip()]

            # Filtro básico: descartar líneas meta (ej. "Claro, ..." o muy cortas)
            claims = []
            for line in raw_lines:
                lower = line.lower()
                if len(line.split()) < 5:
                    continue
                if any(ini in lower for ini in ["claro", "necesito que", "por favor", "cuando me lo proporciones"]):
                    continue
                claims.append(line)

            return claims[:n]

        except NotImplementedError:
            # Fallback de demo: usar frases largas del texto
            sentences = re.split(r"[.!?]", text)
            fallback_claims = [s.strip() for s in sentences if len(s.split()) > 6]
            return fallback_claims[:n]

    def run(self, state: MASState) -> MASState:
        print(f"[{self.name}] Iniciando extracción de afirmaciones para fact-checking...")

        # 1. Validar que haya documentos
        if not state.documents:
            print(f"[{self.name}] No hay documentos, no se puede hacer fact-checking.")
            state.factcheck_results = []
            return state

        # 2. Combinar texto de los documentos (recortado para no saturar)
        combined_parts = []
        for d in state.documents:
            txt = d.get("text", "")
            if txt and txt.strip():
                combined_parts.append(txt[:1500])

        combined = " ".join(combined_parts)

        if not combined.strip():
            print(f"[{self.name}] Texto combinado vacío, no se puede hacer fact-checking.")
            state.factcheck_results = []
            return state

        # 3. Extraer afirmaciones con LLM (o fallback)
        claims = self.extract_claims_with_llm(combined, n=self.max_claims)

        results = []
        for c in claims:
            results.append({
                "claim": c,
                "status": "Requiere verificación adicional",
                "evidence_urls": []   # En esta versión ligera aún no buscamos evidencia
            })

        state.factcheck_results = results
        print(f"[{self.name}] Se identificaron {len(results)} afirmaciones candidatas.")
        return state

#Agente Coordinador / Supervisor

In [ ]:
class CoordinatorAgent(BaseAgent):
    """
    Orquesta todo el pipeline:
    - Búsqueda web
    - Scraping
    - NLP
    - Fact-checking
    - Generación del informe final
    """

    def __init__(self, search_agent, scraper_agent, nlp_agent, fact_agent, name="coordinator"):
        super().__init__(name)
        self.search_agent = search_agent
        self.scraper_agent = scraper_agent
        self.nlp_agent = nlp_agent
        self.fact_agent = fact_agent

    def run(self, state: MASState) -> MASState:
        print(f"\n[{self.name}] Iniciando pipeline multi-agente...\n")

        # 1. Agente de búsqueda
        state = self.search_agent.run(state)

        # 2. Scraper
        state = self.scraper_agent.run(state)

        # Validación crítica: ¿hay documentos?
        if not state.documents:
            print(f"[{self.name}] No se obtuvieron documentos tras el scraping. Pipeline detenido.")
            return state

        # 3. NLP
        state = self.nlp_agent.run(state)

        # 4. Fact-checking
        state = self.fact_agent.run(state)

        print(f"\n[{self.name}] Pipeline completado correctamente.\n")
        return state

    def build_report(self, state: MASState) -> str:
        """
        Usa el LLLM para generar el informe final basado en state.
        """
        print(f"[{self.name}] Generando informe con LLM...")

        prompt = f"""
        Eres un analista de inteligencia de mercado.
        Genera un informe ejecutivo claro y profesional basado en los resultados siguientes:

        Documentos analizados:
        {state.documents}

        Resultados de NLP:
        {state.nlp_results}

        Afirmaciones identificadas para fact-checking:
        {state.factcheck_results}

        Entrega un informe en secciones:
        - Resumen ejecutivo
        - Percepción general de las marcas
        - Temas principales (clusters)
        - Sentimiento por documento o marca
        - Afirmaciones para verificación
        - Conclusiones
        """

        try:
            report = call_llm(prompt)
        except Exception as e:
            report = f"No fue posible generar el informe con LLM. Error: {e}"

        return report

#Informe

In [ ]:
query = "Analiza cómo perciben los medios latinoamericanos a las marcas Samsung, Apple y Xiaomi en reseñas recientes de smartphones."
state = MASState(query=query)

search_agent = SmartphoneWebSearchAgent()
scraper_agent = ScraperAgent()
nlp_agent = SmartphoneNLPAgent(n_clusters=3)
fact_agent = FactCheckAgent()

coordinator = CoordinatorAgent(
    search_agent=search_agent,
    scraper_agent=scraper_agent,
    nlp_agent=nlp_agent,
    fact_agent=fact_agent
)

state = coordinator.run(state)
informe = coordinator.build_report(state)
print(informe)


[coordinator] Iniciando pipeline multi-agente...

[smartphone_search] Se seleccionaron 3 artículos concretos.
[scraper] Descargando: https://www.xataka.com/moviles/samsung-galaxy-s24-ultra-analisis-caracteristicas-precio-especificaciones
[scraper] Descargando: https://www.xataka.com/analisis/apple-iphone-15-pro-max-analisis-caracteristicas-precio-especificaciones
[scraper] Descargando: https://www.xataka.com/moviles/xiaomi-14-analisis-caracteristicas-precio-especificaciones
[scraper] Se obtuvieron 3 documentos con texto suficiente.
[smartphone_nlp] Iniciando análisis NLP sobre 3 documentos...
[smartphone_nlp] Análisis NLP completado.
[fact_checker] Iniciando extracción de afirmaciones para fact-checking...
[fact_checker] Se identificaron 3 afirmaciones candidatas.

[coordinator] Pipeline completado correctamente.

[coordinator] Generando informe con LLM...
## Informe Ejecutivo de Inteligencia de Mercado: Análisis Comparativo de Smartphones Premium (Q1 2024)

**Fecha:** 11 de Junio de 